# Criando DataFrames do Pandas a partir de API
Neste exemplo, usaremos a API do Serviço Geológico dos Estados Unidos (USGS) para obter um objeto JSON de dados de terremotos e convertê-lo em um `pandas.DataFrame`.

API do USGS: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Resposta de 200 significa OK, então podemos extrair os dados do resultado. Como solicitamos à API um payload JSON, podemos extrair isso da resposta com o método `json()`.

### Isolando os Dados da Resposta JSON
Precisamos verificar a estrutura dos dados da resposta para saber onde estão nossos dados.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

A API do USGS fornece informações sobre nossa solicitação na chave `metadata`. Observe que seu resultado será diferente, independentemente do intervalo de datas escolhido, porque a API inclui um carimbo de data/hora para quando os dados foram obtidos.

In [3]:
earthquake_json['metadata']

{'generated': 1717280185000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-05-01&endtime=2024-05-31',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 10082}

Cada elemento na matriz JSON `features` é uma linha de dados para o nosso dataframe.

In [4]:
type(earthquake_json['features'])

list

Seus dados serão diferentes dependendo da data em que você fizer essa solicitação.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.74000001,
  'place': '13 km SSE of Fern Forest, Hawaii',
  'time': 1717113483880,
  'updated': 1717113640310,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv74254316',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv74254316&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 47,
  'net': 'hv',
  'code': '74254316',
  'ids': ',hv74254316,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 28,
  'dmin': 0.03988,
  'rms': 0.300000012,
  'gap': 192,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 1.7 - 13 km SSE of Fern Forest, Hawaii'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.065002441406, 19.3635005950928, 2.61999988555908]},
 'id': 'hv74254316'}

### Converter para DataFrame

Precisamos extrair a seção `properties` de cada entrada na matriz JSON `features` para criar nosso dataframe.

In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.74,"13 km SSE of Fern Forest, Hawaii",1717113483880,1717113640310,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv74254316,",",hv,",",origin,phase-data,",28.0,0.03988,0.3000,192.00,md,earthquake,"M 1.7 - 13 km SSE of Fern Forest, Hawaii"
1,0.30,"27 km NE of Manley Hot Springs, Alaska",1717112855077,1717112957566,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak0246y44iub,",",ak,",",origin,phase-data,",NaN,NaN,0.3700,NaN,ml,earthquake,"M 0.3 - 27 km NE of Manley Hot Springs, Alaska"
2,1.87,"4 km N of Larkfield-Wikiup, CA",1717112708770,1717112890490,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75013532,",",nc,",",focal-mechanism,nearby-cities,origin,phase-data,",39.0,0.05432,0.0700,73.00,md,earthquake,"M 1.9 - 4 km N of Larkfield-Wikiup, CA"
3,1.00,"3 km WSW of Farmers Loop, Alaska",1717112703257,1717112890771,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak0246y440y8,",",ak,",",origin,phase-data,",NaN,NaN,0.6700,NaN,ml,earthquake,"M 1.0 - 3 km WSW of Farmers Loop, Alaska"
4,0.70,"43 km NNE of Beatty, Nevada",1717112388264,1717114258299,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00878395,",",nn,",",origin,phase-data,",14.0,0.07300,0.1474,155.48,ml,earthquake,"M 0.7 - 43 km NNE of Beatty, Nevada"


### (Opcional) Salvar Dados em CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>